In [ ]:
import pandas as pd
from pathlib import Path
from plots.loader import get_subfolders, load_multiple_timings

folders = get_subfolders(Path("./measurements/runs_eiger_16"))
folders += get_subfolders(Path("./measurements/runs_eiger_64"))
# Returns the matrix name and a dictionary of mapping from function to dataframe
matrix, ft = load_multiple_timings(folders, 'nlpkkt240')

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

# SCALING_FACTOR = 10**9 # for seconds
SCALING_FACTOR = 10**6
plt.rcParams['font.family'] = 'Computer Modern Roman'
plt.rcParams['text.usetex'] = True

plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
STYLES = {
    # color, marker, offst, name
    "comb1d": ('deeppink', 'D', (-90, 10), "\\textsc{Comb1D}", None),
    "comb2d": ('dodgerblue', 'o', (-80, 10), "\\textsc{Comb2D}", None),
    "comb3d": ('brown', '^', (-150, -35), "\\textsc{Comb3D}", None),
    "drop_parallel": ('goldenrod', 'h', (-40, -15), "\\textsc{Ours}", None),
    "drop_parallel_1": ('brown', 'h', (-40, -15), "\\textsc{Ours}", None),
    # "todo3": ('brown', '^', (-210, -135), "\\textsc{Tensor Macro}", 26),
    # "todo4": ('darkcyan', 'H', (-75, -15), "\\textsc{Merged}", None),
    # "todo5": ('darkolivegreen', 's', (-120, -118), "\\textsc{Merged+Blocked}", None),
}

In [ ]:
ft.keys()

In [ ]:
fig = plt.figure(figsize=(10, 8), dpi=100)
ax = fig.add_subplot()

keysm = {}
keysm[16] = 0
keysm[64] = 1
# keysm[64] = 0
# keysm[256] = 1
# keysm[100] = 3
# keysm[144] = 4
# keysm[196] = 5

for algo, pretty_name in [('comb1d', 'Comb1D'), ('comb2d', 'Comb2D'), ('comb3d', 'Comb3D'), ('drop_parallel', 'DropParallel')]: #('comb1d', 'Comb1D')
    timings = ft[algo]
    if 'func' not in timings: 
        continue
    func_data = timings[timings['func'] == 'gemm']

    timings = []
    for key, group in func_data.groupby('nodes'): #mpi
        value = group["duration"]
        avg = value.mean()/SCALING_FACTOR
        timings.append({
            "key": key,
            "avg": avg,
            "min": min(value.quantile(0.025)/SCALING_FACTOR, avg),
            "max": max(value.quantile(0.975)/SCALING_FACTOR, avg)
        })

    timings = pd.DataFrame(timings)
    # Calculate the error bars
    eb = [
        (timings['avg'] - timings['min']),
        (timings['max'] - timings['avg'])
    ]

    color, marker, offst, name, rotation = STYLES[algo]

    keys = timings["key"]
    values = timings["avg"]
    ax.annotate(name, (keysm[keys.iloc[-1]], values.iloc[-1]), color=color, xytext=offst, textcoords='offset points', fontsize='x-large', rotation=rotation or 0)
    ax.errorbar([keysm[key] for key in timings["key"]], values, yerr=eb, label=algo, marker=marker, color=color)

    ax.set_xlabel("Cores")
    ax.set_ylabel('Time [ms]',
            rotation='horizontal',
            loc='top')
    ax.yaxis.set_label_coords(0.07, 1.015) 
    ax.set_xticks(range(len(keys)), labels=[f"\\begin{{center}}{mpi}~\\texttt{{nodes}}\\\\{mpi * 36}\\end{{center}}"for mpi in keys])

ax.tick_params(axis='both', direction='in', which='major', pad=5)
ax.grid(which='major', axis='y', linewidth=.5, dashes=(3,3))
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')
plt.yscale('log', base=2)
from matplotlib.ticker import FuncFormatter
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f'{y:.0f}'))

ax.set_title("Squaring computation on \\texttt{" + matrix + "}", fontsize=15)

In [ ]:
fig.savefig("runtime_2.pdf", bbox_inches="tight")